In [144]:
import numpy as np
import math
import pandas as pd
#import seaborn as sns
#import matplotlib.pyplot as plt
import json
#
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import r2_score
from scipy.stats import truncnorm
#
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
#
import xgboost as xgb
#
#
import nevergrad as ng
import autograd.numpy as au
from autograd import grad, jacobian
#
from concurrent import futures
import time
#
from scipy.optimize import Bounds
from scipy.optimize import minimize


from more_itertools import grouper
import cobyqa

# Setup parameter

In [176]:
np.random.RandomState(1234)
np.random.seed(42)

# chemical component to achieve: C 1,55 / Si 0,4 / Mn 0,3 / Cr 11,8 / Mo 0,75 / V 0,82
chemi_component = (np.array([1.55,0.4,0.3,11.8,0.75,0.82]) / 100.0).astype(np.float32)

chemi_names = ['C','Si','Mn','Cr','Mo','V']

# total weight for the final steel production
total_weight = 10_000

total_chemi_to_achieve = total_weight * chemi_component

In [148]:
df = pd.read_csv("data/df_12379.csv")
df_chemi = pd.read_excel("data/chemi.xlsx", engine="openpyxl")
df_price = pd.read_excel("data/scrap.xlsx", engine="openpyxl")

In [149]:
# function to return all information from df
def df_columns_name(df):
    columns = df.columns.tolist()
    remove_columns = ['HeatID', 'HeatOrderID','Energy']
    features_columns = [x for x in columns if x not in remove_columns]
    
    # constant process parameter, start with "Feature"
    constant_features_names = [x for x in features_columns if "Feature" in x]
    
    #"K" means Kreislauf, "L" means Legierungen, "F" means Fremdschrotte, we only want to optimize "F"
    schrotte_features_names = [x for x in features_columns if "Feature" not in x]
    
    # schrotte name for Kreislauf
    kreislauf_schrotte_names = [x for x in features_columns if "K" in x]
    
    # schrotte name for legierung
    legierung_schrotte_names = [x for x in features_columns if "L" in x]
    
    # schrotte name for Fremdschrotte
    fremd_schrotte_names = [x for x in features_columns if "F" in x and len(x) < 4]
    
    return constant_features_names, schrotte_features_names, kreislauf_schrotte_names,legierung_schrotte_names,fremd_schrotte_names

In [162]:
# function to calculate the chemical component weights
# randomly choose a row from df

constant_features_names, schrotte_features_names, kreislauf_schrotte_names, legierung_schrotte_names,fremd_schrotte_names = df_columns_name(df)

def calculate_chemi_component(df, df_chemi):
    
    """
    return the randomly chosen row, and its constant column, kreislauf column and 
    legierung column, use them to return the chemical component of fremdschrotte column
    """
    df_random_row = df.sample()
    
    # calculate the constant features
    constant_column = (df_random_row[constant_features_names].values[0]).astype(np.float32)
    
    # calculate the chemical component for kreislauf
    kreislauf_column = (df_random_row[kreislauf_schrotte_names].values[0]).astype(np.float32)
    kreislauf_chemical_table = df_chemi[chemi_names].iloc[len(kreislauf_schrotte_names)-1:]
    chemi_component_kreislauf = (np.dot(kreislauf_column, kreislauf_chemical_table) /100.0).astype(np.float32)
    
    # calculate the chemical component for legierungen
    legierung_column = (df_random_row[legierung_schrotte_names].values[0]).astype(np.float32)
    legierung_chemical_table = df_chemi[chemi_names].iloc[len(fremd_schrotte_names):len(kreislauf_schrotte_names)-1]
    chemi_component_legierung = (np.dot(legierung_column, legierung_chemical_table) /100.0).astype(np.float32)
    
    # calculate the chemical compoent for fremdschrotte
    chemi_to_achieve_fremdschrotte = (np.abs(total_chemi_to_achieve - chemi_component_kreislauf - chemi_component_legierung)).astype(np.float32)
    
    return constant_column, kreislauf_column, legierung_column, chemi_to_achieve_fremdschrotte

In [164]:
 constant_column, kreislauf_column, legierung_column, chemi_to_achieve_fremdschrotte = calculate_chemi_component(df, df_chemi)

In [163]:
# construct the chemical table
# assume that every company's chemical elements for every schrott is identical
def fremdschrott_chemi_table(df_chemi):
    df_chemi_fremdschrott= df_chemi[chemi_names].iloc[:len(fremd_schrotte_names)]
    fremdschrott_chemi = df_chemi_fremdschrott.T
    n_times = 9
    temp_dfs = []
    
    for col_name in fremdschrott_chemi.columns:
        temp_df = fremdschrott_chemi[[col_name]].copy()
        for i in range(1, n_times+1):
            temp_df[f'{col_name}{i}'] = fremdschrott_chemi[col_name]
        temp_dfs.append(temp_df)

    fremdschrotte_chemi_table = pd.concat(temp_dfs, axis=1) / 100.0
    
    return fremdschrotte_chemi_table

# Opt

In [178]:
length_fremdschrott = len(fremd_schrotte_names)
length_company = 10

total_variable_length = length_fremdschrott * length_company

df_price_list = df_price["price"].to_numpy()
price_list = df_price_list[:total_variable_length]


x_lower = np.zeros(total_variable_length)
x_upper = np.ones(total_variable_length) * total_weight


fremdschrotte_chemi_table = fremdschrott_chemi_table(df_chemi)
# right hand side of equality constraints
aeq = np.array(fremdschrotte_chemi_table)

# start values
x_start = np.zeros(total_variable_length)

# scipy specific
bounds = Bounds([0.0]*total_variable_length, [max(total_chemi_to_achieve)]*total_variable_length)

# max iteration
max_iter = 500 #1000

## COBYQA + ANN

In [167]:
# load xgboost and ann models
xgb_model = xgb.Booster()
xgb_models = xgb_model.load_model("models/XGB.json")

ann_model = tf.keras.models.load_model("models/ANN")

In [168]:
# function for xgb prediction
def f_xgb(x):
    y = xgb_model.predict(xgb.DMatrix([x]))   #x1.reshape(1,-1))
    return y.item()

# function to calculate the total cost of xgboost version
def sum_t3_xgb(x):
    """return sum of three objectives"""
    summe = 0
    quantity = np.array([sum(g) for g in list(grouper(10, x))])
    for q in quantity:
        if q <= 10.0:
            summe += 0.0
        elif q > 10.0 and q <= 50.0:
            summe += 2.5 * (q-10) 
        else:
            summe += 100.0

        return summe

# objective xgboost
def objective(x, constant_column, kreislauf_column, legierung_column):
    t1 = np.dot(x, price_list)
    list_fremdschrotte = [sum(g) for g in list(grouper(10, x))]
    features = np.concatenate((constant_column, kreislauf_column, list_fremdschrotte, legierung_column))
    t2 = f_xgb(features)
    t3 = sum_t3_xgb(x)
    return (t1 + t2 + t3).item()

# ann prediction 
@tf.function
def tf_ann(x,constant_column,kreislauf_column,legierung_column):
    x = tf.convert_to_tensor(x, dtype=tf.float32)
    constant_column_tf = tf.convert_to_tensor(constant_column, dtype=tf.float32)
    kreislauf_column_tf = tf.convert_to_tensor(kreislauf_column, dtype=tf.float32)
    legierung_column_tf = tf.convert_to_tensor(legierung_column, dtype=tf.float32)

    list_fremdschrotte = tf.reduce_sum(tf.reshape(x,[-1,10]), axis=1)#tf.convert_to_tensor([sum(g) for g in list(grouper(10, x))])
    x = tf.concat([constant_column_tf, kreislauf_column_tf, list_fremdschrotte, legierung_column_tf], axis=0)
    x = tf.convert_to_tensor([x])
    ann_pred = ann_model(x)[0]
    return ann_pred

# function to calculate the logistic cost of tf version
@tf.function
def sum_t3_tf(x):
    """
    X: tf.Tensor, the quantity of every scrap
    """
    summe = tf.constant(0.0, dtype=tf.float32)
    quantity = tf.reduce_sum(tf.reshape(x,[-1,10]), axis=1) #tf.convert_to_tensor([sum(g) for g in list(grouper(10, x))])

    for q in quantity:
        q = tf.reshape(q, ())
        summe += tf.cond(q <= 10.0, 
                         lambda: 0.0, 
                         lambda: tf.cond(q > 10.0 and q <= 50.0, 
                                         lambda: 2.5 * (q - 10), 
                                         lambda: 100.0))
    return summe


# function for calculate the total cost, tf version
@tf.function
def objective_tf(x,constant_column,kreislauf_column,legierung_column):
    x = tf.convert_to_tensor(x, dtype=tf.float32)
    price_list_tf = tf.convert_to_tensor(price_list, dtype=tf.float32)
    t1 = tf.tensordot(x, price_list_tf,axes=1)
    t2 = tf_ann(x,constant_column,kreislauf_column,legierung_column)
    t3 = sum_t3_tf(x)
    
    return t1 + t2 + t3

# function to calculate the jacobian of tf 
@tf.function
def grad_f_ann_tf(x,constant_column,kreislauf_column,legierung_column):
    x1 = tf.convert_to_tensor(x, dtype=tf.float32)
    with tf.GradientTape() as tape:
        tape.watch(x1)
        t2 = objective_tf(x1,constant_column,kreislauf_column,legierung_column)
        y = tape.jacobian(t2, x1)
        return y

In [169]:
x = np.random.randn(160).astype(np.float32)

constant_column, kreislauf_column, legierung_column, \
    chemi_to_achieve_fremdschrotte = calculate_chemi_component(df, df_chemi)

In [171]:
tf_ann(x,constant_column,kreislauf_column,legierung_column)

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([3739.718], dtype=float32)>

In [172]:
objective_tf(x,constant_column,kreislauf_column,legierung_column)

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([3743.38], dtype=float32)>

In [173]:
grad_f_ann_tf(x,constant_column,kreislauf_column,legierung_column)

2023-04-25 09:47:52.459250: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/add_grad/Reshape/pfor/concat/loop_body/PartitionedCall/pfor/Reshape' with dtype int32 and shape [1]
	 [[{{node gradients/add_grad/Reshape/pfor/concat/loop_body/PartitionedCall/pfor/Reshape}}]]
2023-04-25 09:47:52.460809: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/add_grad/Reshape/pfor/concat/loop_body/PartitionedCall/pfor/Reshape' with dtype int32 and shape [1]
	 [[{{node gradients/add_grad/Reshape/pfor/concat/loop_body/PartitionedCall/pfor/Reshape}}]]
2023-04-25 09:47:52.464298: I tensorflow/core/common_runtime/executo

<tf.Tensor: shape=(1, 160), dtype=float32, numpy=
array([[0.28, 0.26, 0.29, 0.27, 0.25, 0.3 , 0.28, 0.26, 0.28, 0.25, 0.31,
        0.3 , 0.29, 0.28, 0.28, 0.31, 0.28, 0.31, 0.29, 0.3 , 0.45, 0.45,
        0.42, 0.43, 0.42, 0.43, 0.44, 0.42, 0.41, 0.42, 1.5 , 1.6 , 1.5 ,
        1.4 , 1.5 , 1.6 , 1.5 , 1.4 , 1.4 , 1.3 , 0.58, 0.6 , 0.6 , 0.57,
        0.6 , 0.56, 0.6 , 0.57, 0.57, 0.56, 1.9 , 2.  , 2.1 , 2.1 , 2.  ,
        2.1 , 2.1 , 2.  , 1.9 , 2.1 , 1.2 , 1.2 , 1.2 , 1.2 , 1.3 , 1.2 ,
        1.3 , 1.1 , 1.2 , 1.1 , 0.68, 0.69, 0.68, 0.7 , 0.69, 0.67, 0.67,
        0.69, 0.7 , 0.7 , 0.87, 0.88, 0.84, 0.83, 0.87, 0.86, 0.83, 0.84,
        0.87, 0.87, 0.2 , 0.21, 0.23, 0.22, 0.2 , 0.21, 0.21, 0.2 , 0.21,
        0.21, 5.9 , 6.5 , 6.4 , 6.3 , 6.5 , 5.6 , 6.2 , 6.1 , 5.6 , 6.3 ,
        0.21, 0.2 , 0.19, 0.22, 0.19, 0.21, 0.2 , 0.2 , 0.22, 0.21, 0.73,
        0.75, 0.77, 0.71, 0.75, 0.72, 0.72, 0.78, 0.77, 0.77, 0.37, 0.36,
        0.35, 0.37, 0.35, 0.38, 0.37, 0.39, 0.38, 0.38, 5.  , 

In [196]:
def optimize_cobyqa(constant_column, kreislauf_column, legierung_column, beq):
    # Wrap the objective function with a lambda to include the constant variables
    wrapped_objective = lambda x: objective(x, constant_column, kreislauf_column, legierung_column)
    
    start_cobyqa = time.time()
    res_cobyqa = cobyqa.minimize(wrapped_objective, x0=x_start, xl=x_lower, xu=x_upper,
                                  aeq=aeq, beq=beq, options={"disp": False, "maxiter": max_iter})
    end_cobyqa = time.time()

    elapsed_time_cobyqa = end_cobyqa - start_cobyqa
    
    c_violation = (np.dot(aeq, res_cobyqa.x) - beq).tolist()

    return res_cobyqa.x, res_cobyqa.fun, c_violation, elapsed_time_cobyqa


In [197]:
def optimize_grad(constant_column, kreislauf_column, legierung_column, beq):
    # Wrap the objective and gradient functions with lambda functions
    
    wrapped_objective_tf = lambda x: objective_tf(x.astype(np.float32), constant_column, kreislauf_column, legierung_column)
    wrapped_grad_f_ann_tf = lambda x: grad_f_ann_tf(x.astype(np.float32), constant_column, kreislauf_column, legierung_column)
        
    def equality_fun(x):
        return np.dot(aeq, x) - beq 
    
    eq_cons = {'type': 'eq','fun' : equality_fun}
    
    start = time.time()
    result = minimize(fun=wrapped_objective_tf, x0=x_start, jac=wrapped_grad_f_ann_tf, constraints=[eq_cons], method='SLSQP',
                      options={'disp': False, 'maxiter':max_iter, 'ftol':1e-9}, bounds=bounds, tol=1e-9)

    
    end = time.time()
    c_violation = (np.dot(aeq, result.x) - beq).tolist()
    elapsed_time = end - start
    return result.x, result.fun, c_violation, elapsed_time

In [ ]:
results = []
for i in range(100):
    print(f"################# Optimizing for {i} COBYQA iteration #################")
    constant_column, kreislauf_column, legierung_column, \
    chemi_to_achieve_fremdschrotte = calculate_chemi_component(df, df_chemi)
    
    beq = chemi_to_achieve_fremdschrotte
    
    results_current = {'i': i, 'beq': beq.tolist()}

    
    ############### cobyqa ##########################

    x, loss, c_violation, elapsed_time_cobyqa = optimize_cobyqa(constant_column, kreislauf_column, legierung_column, beq)
    results_current['cobyqa'] = {}
    results_current['cobyqa']['x_min'] = x.tolist()
    results_current['cobyqa']['loss'] = loss
    results_current['cobyqa']['c_violation'] = c_violation
    results_current['cobyqa']['time'] = elapsed_time_cobyqa
    ###################################################
    
    ############## ann ###############################
    
    print(f"################# Optimizing for {i} ann iteration #################")
    x_ann, loss_ann, c_violation_ann, elapsed_time_ann = optimize_grad(constant_column, kreislauf_column, legierung_column, beq)
    results_current['ann'] = {}
    results_current['ann']['x_min'] = x_ann.tolist()
    results_current['ann']['loss'] = objective(x_ann,constant_column, kreislauf_column, legierung_column)
    results_current['ann']['c_violation'] = c_violation_ann
    results_current['ann']['time'] = elapsed_time_ann
    
    if np.sum(np.abs(c_violation_ann)) / np.sum(beq) > 0.05:
        results_current['ann']['is_x_accept'] = 'no'
    else:
        results_current['ann']['is_x_accept'] = 'yes'
    
    results.append(results_current)
    with open('optimization/results2.json', 'w') as file:
        json.dump(results, file)
     

################# Optimizing for 0 COBYQA iteration #################
################# Optimizing for 0 ann iteration #################
################# Optimizing for 1 COBYQA iteration #################
################# Optimizing for 1 ann iteration #################
################# Optimizing for 2 COBYQA iteration #################
################# Optimizing for 2 ann iteration #################
################# Optimizing for 3 COBYQA iteration #################
################# Optimizing for 3 ann iteration #################
################# Optimizing for 4 COBYQA iteration #################
################# Optimizing for 4 ann iteration #################
################# Optimizing for 5 COBYQA iteration #################
################# Optimizing for 5 ann iteration #################
################# Optimizing for 6 COBYQA iteration #################
################# Optimizing for 6 ann iteration #################
################# Optimizing for 7 COBYQA